In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate


In [2]:
async def extract_text(chain):
    collected_text = []
    async for event in chain.astream_events({"topic": "parrot"}):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        print(text, end="", flush=True)
    return "".join(collected_text)



In [3]:
#bedrock client 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # replace with your region
)

In [4]:
# langchain과 bedrock client 연결하여 langchain interface안에서 bedrock api 호출 가능하게함
llm = ChatBedrockConverse(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    temperature=0.7,
    max_tokens=2000
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "태양의 온도에 대해 말해줘")
])

chain = prompt | llm | StrOutputParser()

In [5]:
full_text = await extract_text(chain)

태양의 온도는 매우 높습니다. 태양 중심부의 온도는 약 1,500만도 정도로 추정됩니다. 이는 지구 표면 온도의 약 300배에 달하는 엄청난 고온입니다. 

양 표면인 광구 온도는 약 5,800K(kelvin, 절대온도 단위) 정도로 알려져 있습니다. 이는 섭씨 약 5,500도에 해당합니다.

양의 고온은 핵융합 반응에 의해 발생합니다. 태양 중심부에서는 수소 원자핵들이 엄청난 중력과 압력에 의해 융합되면서 헬륨으로 바뀌고, 이 과정에서 막대한 에너지가 방출됩니다. 이 에너지가 열과 빛의 형태로 방출되어 태양의 고온을 유지하게 됩니다.

양의 이런 고온 덕분에 지구에 생명체가 살 수 있는 적정 온도를 유지할 수 있습니다. 태양은 지구의 생명 활동에 필수적인 에너지원입니다.

In [12]:
print("langchain 방삭") 


langchain 방삭


In [15]:
#input { json 포맷 }
input_format = {
    "relevant_information" : "상사병은 상사로 인한 스트레스가 쌓인 병을 뜻한다", 
    "question" : "상사병은 무엇인가요?"
    
}

In [16]:
#prompt(text로 변환) input의 json형식에서 relevant_information와 question 정보를 담아서 prompt로 보내면 prompt의 relevant_information과 question 와 연결된다
template_general = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

Relevant Information:
{relevant_information}

Conversation:
Human: {question}
AI:"""

prompt_general  = PromptTemplate( input_variables=["relevant_information", "question"], template=template_general)

In [17]:
#위에 bedrock과 langchain interface를 연동한 object
llm

ChatBedrockConverse(client=<botocore.client.BedrockRuntime object at 0x7f6c7f274500>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', max_tokens=2000, temperature=0.7, provider='anthropic', supports_tool_choice_values=('auto', 'any', 'tool'))

In [18]:
#llm의 결과를 string으로 변환해서 보내주는 parser
parser = StrOutputParser()


In [19]:
#langchain문법을 통해 연결한다  input { json 포맷 }  -> prompt(text로 변환) -> llm(bedrock) -> output(string타입, json타입)
langchain = prompt_general | llm | parser

In [20]:
langchain.invoke(input_format)

'상사병은 직장 내에서 상사로부터 받는 스트레스나 갈등으로 인해 발생하는 심리적, 신체적 증상을 일컫는 말입니다. 주요 증상으로는 불면증, 두통, 소화기 장애, 우울증 등이 있습니다. 상사의 부당한 지시나 인격 모독, 괴롭힘 등으로 인해 발생할 수 있습니다.\n\n상사병을 예방하기 위해서는 상사와 원활한 의사소통을 하고, 업무 스트레스를 적절히 해소할 수 있는 방법을 찾는 것이 중요합니다. 회사 차원에서도 상사 교육, 고충 처리 제도 등을 통해 직원들의 스트레스를 관리해 줄 필요가 있죠.\n\n상사병이 지속되면 결국 직장을 그만둬야 하는 상황이 올 수 있으므로, 초기에 대처하는 것이 중요합니다. 상사와의 관계 개선이 어렵다면 인사 부서나 고충 상담실 등을 적극 활용하는 것도 하나의 방법입니다.'